<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Hands on with `pandas`: Part 2

---

- **Objective:** Know how to handle null and missing values.

Sometimes, values will be missing from the source data or as a byproduct of manipulations. It is very important to detect missing data. Missing data can:

- Make the entire row ineligible to be training data for a model.
- Hint at data-collection errors.
- Indicate improper conversion or manipulation.
- Actually not be missing — it sometimes means "zero," "false," "not applicable," or "entered an empty string."

For example, a `.csv` file might have a missing value in some data fields:

```
tool_name,material,cost
hammer,wood,8
chainsaw,,
wrench,metal,5
```

When this data is imported, "null" values will be stored in the second row (in the "material" and "cost" columns).

> In Pandas, a "null" value is either `None` or `np.NaN` (Not a Number). Many fixed-size numeric datatypes (such as integers) do not have a way of representing `np.NaN`. So, numeric columns will be promoted to floating-point datatypes that do support it. For example, when importing the `.csv` file above:

> - **For the second row:** `None` will be stored in the "material" column and `np.NaN` will be stored in the "cost" column. The entire "cost" column (stored as a single `ndarray`) must be stored as floating-point values to accommodate the `np.NaN`, even though an integer `8` is in the first row.

In [ ]:
import pandas as pd

drinks = pd.read_csv("data/drinks.csv")

In [ ]:
# Missing values are usually excluded in calculations by default.
drinks["continent"].value_counts()              # Excludes missing values in the calculation

In [ ]:
# Includes missing values
drinks["continent"].value_counts(dropna=False)

In [ ]:
# Find missing values in a Series.
# True if missing, False if not missing
drinks["continent"].isnull()

In [ ]:
# Count the missing values — sum() works because True is 1 and False is 0.
drinks["continent"].isnull().sum()

In [ ]:
# True if not missing, False if missing
drinks["continent"].notnull()

In [ ]:
# Only show rows where continent is not missing.
drinks[drinks["continent"].notnull()]

**Understanding Pandas Axis**

In [ ]:
# Sums "down" the 0 axis (rows) — so, we get the sums of each column
drinks.sum(axis=0)

In [ ]:
# axis=0 is the default.
drinks.sum()

In [ ]:
# Sums "across" the 1 axis (columns) — so, we get the sums of numeric values in the row (beer+spirit+wine+liters+…)
drinks.sum(axis=1)

**Find missing values in a `DataFrame`.**

In [ ]:
# Count the missing values in each column — remember by default, axis=0.
print((drinks.isnull().sum()))

**Dropping Missing Values**

In [ ]:
# Drop a row if ANY values are missing from any column — can be dangerous!
drinks.dropna()

In [ ]:
# Drop a row only if ALL values are missing.
drinks.dropna(how='all')

**Filling Missing Values**<br>
You may have noticed that the continent North America (NA) does not appear in the `continent` column. Pandas read in the original data and saw "NA", thought it was a missing value, and converted it to a `NaN`, missing value.

In [ ]:
# Fill in missing values with "NA" — this is dangerous to do without manually verifying them!
drinks["continent"].fillna(value='NA')

In [ ]:
# Modifies "drinks" in-place
drinks["continent"].fillna(value='NA', inplace=True)

In [ ]:
# Turn off the missing value filter — this is a better approach!
drink_cols = ['country', 'beer', 'spirit', 'wine', 'litres', 'continent']
drinks = pd.read_csv('data/drinks.csv', header=0, names=drink_cols, na_filter=False)

<a id="exercise-three"></a>
### Exercise 3

In [ ]:
# Read ufo.csv into a DataFrame called "ufo".
ufo_data = 'data/ufo.csv'
ufo = pd.read_csv(ufo_data)

In [ ]:
# Check the shape of the DataFrame.

In [ ]:
# What are the three most common colors reported?

In [ ]:
# Rename any columns with spaces so that they don't contain spaces.

In [ ]:
# For reports in VA, what's the most common city?

In [ ]:
# Print a DataFrame containing only reports from Arlington, VA.

In [ ]:
# Count the number of missing values in each column.

In [ ]:
# How many rows remain if you drop all rows with any missing values?

<a id="split-apply-combine"></a>
### Split-Apply-Combine

Split-apply-combine is a pattern for analyzing data. Suppose we want to find mean beer consumption per country. Then:

- **Split:** We group data by continent.
- **Apply:** For each group, we apply the `mean()` function to find the average beer consumption.
- **Combine:** We now combine the continent names with the `mean()`s to produce a summary of our findings.

In [ ]:
# For each continent, calculate the mean beer servings.
drinks.groupby('continent')["beer"].mean()

In [ ]:
# For each continent, calculate the mean of all numeric columns.
drinks.groupby('continent').mean()

In [ ]:
# For each continent, describe beer servings.
drinks.groupby('continent')["beer"].describe()

In [ ]:
# Similar, but outputs a DataFrame and can be customized — "agg" allows you to aggregate results of Series functions
drinks.groupby('continent')["beer"].agg(['count', 'mean', 'min', 'max'])
drinks.groupby('continent')["beer"].agg(['count', 'mean', 'min', 'max']).sort_values('mean')

In [ ]:
# For each continent, describe all numeric columns.
drinks.groupby('continent').describe()

In [ ]:
# For each continent, count the number of rows.
print((drinks.groupby('continent')["continent"].count()))
print((drinks.continent.value_counts()))   # should be the same

<a id="exercise-four"></a>
### Exercise 4

Use the "users" `DataFrame` or "users" file in the Data folder to complete the following.

In [ ]:
users = pd.read_table('data/user.tbl', sep='|')

In [ ]:
# For each occupation in "users", count the number of occurrences.

In [ ]:
# For each occupation, calculate the mean age.

In [ ]:
# For each occupation, calculate the minimum and maximum ages.

In [ ]:
# For each combination of occupation and gender, calculate the mean age.

----

<a id="joining-dataframes"></a>
### Joining (Merging) `DataFrames`

In [ ]:
movie_cols = ['movie_id', 'title']
movies_filename = 'data/movies.tbl'

movies = pd.read_table(movies_filename,
                       sep='|',
                       header=None,
                       names=movie_cols,
                       usecols=[0, 1],
                       encoding="latin-1")
movies.head()

In [ ]:
rating_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_filename = 'data/movie_ratings.tsv'

ratings = pd.read_table(ratings_filename, sep='\t', header=None, names=rating_cols)
ratings.head()

Merge "movies" and "ratings" (inner join on "movie_id")

- if you leave `on` blank, `pandas` will try and work out common columns - but it's better to be explicit!
- `how` defaults to "inner"

In [ ]:
movie_ratings = pd.merge(movies, ratings, on="movie_id", how="inner")
movie_ratings.head()

In [ ]:
print(movies.shape)
print(ratings.shape)
print(movie_ratings.shape)

<a id="multiple-columns"></a>
### Selecting Multiple Columns and Filtering Rows

In [ ]:
# Select multiple columns — yet another overload of the DataFrame indexing operator!
my_cols = ['City', 'State']     # Create a list of column names...
ufo[my_cols]                    # ...and use that list to select columns.

In [ ]:
# Or, combine into a single step (this is a Python list inside of the Python index operator!).
ufo[['City', 'State']]

**Use `loc` to select columns by name.**

In [ ]:
# "loc" locates the values from the first parameter (colon means "all rows"), and the column "City".
ufo.loc[:, 'City']  

In [ ]:
# Select two columns.
ufo.loc[:, ['City', 'State']]

In [ ]:
# Select a range of columns — unlike Python ranges, Pandas index ranges INCLUDE the final column in the range.
ufo.loc[:, 'City':'State']

In [ ]:
# "loc" can also filter rows by "name" (the index).
# Row 0, all columns
ufo.loc[0, :]

In [ ]:
# Rows 0/1/2, all columns
ufo.loc[0:2, :]

In [ ]:
# Rows 0/1/2, range of columns
ufo.loc[0:2, 'City':'State'] 

In [ ]:
# Use "iloc" to filter rows and select columns by integer position.
# (Remember that rows/columns use indices, so "iloc" lets you refer to indices via their index rather than value!)
# All rows, columns in position 0/3 (City/State)
ufo.iloc[:, [0, 3]]

In [ ]:
# All rows, columns in position 0/1/2/3
# Note here it is NOT INCLUDING 4 because this is an integer range, not a Pandas index range!
ufo.iloc[:, 0:4]

In [ ]:
# Rows in position 0/1/2, all columns
ufo.iloc[0:3, :] 

<a id="other-features"></a>
### OPTIONAL: Other Commonly Used Features

In [ ]:
# Apply an arbitrary function to each value of a Pandas column, storing the result in a new column.
users['under30'] = users["age"].apply(lambda age: age < 30)

In [ ]:
# Apply an arbitrary function to each row of a DataFrame, storing the result in a new column.
#  (Remember that, by default, axis=0. Since we want to go row by row, we set axis=1.)
users['under30male'] = users.apply(lambda row: row["age"] < 30 and row["gender"] == 'M', axis=1)

In [ ]:
# Map existing values to a different set of values.
users['is_male'] = users["gender"].map({'F':0, 'M':1})

In [ ]:
# Replace all instances of a value in a column (must match entire value).
ufo["State"].replace('Fl', 'FL', inplace=True)

In [ ]:
# String methods are accessed via "str".
ufo["State"].str.upper()                               # Converts to upper case
# checks for a substring
ufo['Colors Reported'].str.contains('RED', na='False') 

In [ ]:
# Convert a string to the datetime format (this is often slow — consider doing it in the "read_csv()" method.)
ufo["Time"] = pd.to_datetime(ufo["Time"])
ufo["Time"].dt.hour                        # Datetime format exposes convenient attributes
(ufo["Time"].max() - ufo["Time"].min()).days  # Also allows you to do datetime "math"

In [ ]:
# Change the datatype of a column.
drinks['beer'] = drinks["beer"].astype('float')

In [ ]:
# Create dummy variables for "continent" and exclude first dummy column.
continent_dummies = pd.get_dummies(drinks["continent"], prefix='cont').iloc[:, 1:]
continent_dummies

In [ ]:
# Concatenate two DataFrames (axis=0 for rows, axis=1 for columns).
pd.concat([drinks, continent_dummies], axis=1)

<a id="uncommon-features"></a>
### OPTIONAL: Other Less-Used Features of Pandas

In [ ]:
# Detecting duplicate rows
users.duplicated()          # True if a row is identical to a previous row

In [ ]:
users.duplicated().sum()    # Count of duplicates

In [ ]:
users[users.duplicated()]   # Only show duplicates

In [ ]:
users.drop_duplicates()     # Drop duplicate rows

In [ ]:
users["age"].duplicated()      # Check a single column for duplicates

In [ ]:
users.duplicated(['age', 'gender', 'zip_code']).sum()   # Specify columns for finding duplicates

In [ ]:
# Convert a range of values into descriptive groups.
drinks['beer_level'] = 'low'    # Initially set all values to "low"
drinks.loc[drinks["beer"].between(101, 200), 'beer_level'] = 'med'     # Change 101-200 to "med"
drinks.loc[drinks["beer"].between(201, 400), 'beer_level'] = 'high'    # Change 201-400 to "high"

In [ ]:
# Display a cross-tabulation of two Series.
pd.crosstab(drinks["continent"], drinks["beer_level"])

In [ ]:
# Convert "beer_level" into the "category" datatype.
drinks['beer_level'] = pd.Categorical(drinks["beer_level"], categories=['low', 'med', 'high'])
drinks.sort_values('beer_level')   # Sorts by the categorical ordering (low to high)

In [ ]:
# Limit which rows are read when reading in a file — useful for large files!
pd.read_csv('data/drinks.csv', nrows=10)           # Only read first 10 rows
pd.read_csv('data/drinks.csv', skiprows=[1, 2])    # Skip the first two rows of data

In [ ]:
# Write a DataFrame out to a .csv
drinks.to_csv('drinks_updated.csv')                 # Index is used as first column
drinks.to_csv('drinks_updated.csv', index=False)    # Ignore index

In [ ]:
# Create a DataFrame from a dictionary.
pd.DataFrame({'capital':['Montgomery', 'Juneau', 'Phoenix'], 'state':['AL', 'AK', 'AZ']})

In [ ]:
# Create a DataFrame from a list of lists.
pd.DataFrame([['Montgomery', 'AL'], ['Juneau', 'AK'], ['Phoenix', 'AZ']], columns=['capital', 'state'])

In [ ]:
# Randomly sample a DataFrame.
import numpy as np
mask = np.random.rand(len(drinks)) < 0.66   # Create a Series of Booleans
train = drinks[mask]                        # Will contain around 66% of the rows
test = drinks[~mask]                        # Will contain the remaining rows

In [ ]:
# Change the maximum number of rows and columns printed ('None' means unlimited).
pd.set_option('max_rows', None)     # Default is 60 rows
pd.set_option('max_columns', None)  # Default is 20 columns
print(drinks)

In [ ]:
# Reset options to defaults.
pd.reset_option('max_rows')
pd.reset_option('max_columns')

In [ ]:
# Change the options temporarily (settings are restored when you exit the "with" block).
with pd.option_context('max_rows', None, 'max_columns', None):
    print(drinks)

<a id="summary"></a>
### Summary

Believe it or not, we've only barely touched the surface of everything that Pandas offers. Don't worry if you don't remember most of it — for now, just knowing what exists is key. Remember that the more you use Pandas to manipulate data, the more of these functions you will take interest in, look up, and remember.

In these notebooks, the most important things to familiarise yourself with are the basics:
- Manipulating `DataFrames` and `Series`
- Filtering columns and rows
- Handling missing values
- Split-apply-combine (this one takes some practice!)